# Classificador de Texto: AI vs Human
Este notebook treina um modelo para distinguir textos gerados por inteligência artificial e humanos.

In [1]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, Embedding, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import initializers

## 1. Definir semente para garantir reprodutibilidade

In [2]:
os.environ['TF_DETERMINISTIC_OPS'] = '1'
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

## 2. Carregar os dados de treino, validação e teste

In [3]:
def load_dataset(path, sep='\t'):
    return pd.read_csv(path, sep=sep, encoding='utf-8')

X_train = load_dataset("../data/test_input.csv")
y_train = load_dataset("../data/test_output.csv")

X_val = load_dataset("../data/human_ai_input.csv")
y_val = load_dataset("../data/human_ai_output.csv")

X_test = load_dataset("../data/dataset3_inputs.csv")
ids = X_test["ID"]

## 3. Tokenização dos textos para vetorizar as frases

In [22]:
max_words = 20000
max_len = 500

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train["Text"])

def tokenize_pad(texts):
    seq = tokenizer.texts_to_sequences(texts)
    return pad_sequences(seq, maxlen=max_len)

X_train_pad = tokenize_pad(X_train["Text"])
X_val_pad = tokenize_pad(X_val["Text"])
X_test_pad = tokenize_pad(X_test["Text"])

## 4. Converter rótulos para 0 (Human) e 1 (AI)

In [23]:
y_train = y_train["Label"].map({"AI": 1, "Human": 0}).values
y_val = y_val["Label"].map({"AI": 1, "Human": 0}).values

## 5. Criar o modelo de rede neural

In [24]:
embedding_dim = 128
model = Sequential([
    Input((max_len,)),
    Embedding(max_words, embedding_dim, embeddings_initializer=initializers.GlorotUniform(seed=44)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## 6. Treinar o modelo

In [25]:
history = model.fit(X_train_pad, y_train, epochs=15, batch_size=64,
                    validation_data=(X_val_pad, y_val), verbose=1)

Epoch 1/15
47/47 [==============================] - 4s 79ms/step - loss: 0.4076 - accuracy: 0.8043 - val_loss: nan - val_accuracy: 0.1665
Epoch 2/15
47/47 [==============================] - 3s 73ms/step - loss: 0.0023 - accuracy: 0.9997 - val_loss: nan - val_accuracy: 0.1667
Epoch 3/15
47/47 [==============================] - 3s 68ms/step - loss: 6.9537e-04 - accuracy: 1.0000 - val_loss: nan - val_accuracy: 0.1667
Epoch 4/15
47/47 [==============================] - 3s 73ms/step - loss: 2.6423e-04 - accuracy: 1.0000 - val_loss: nan - val_accuracy: 0.1667
Epoch 5/15
47/47 [==============================] - 3s 72ms/step - loss: 1.9928e-04 - accuracy: 1.0000 - val_loss: nan - val_accuracy: 0.1667
Epoch 6/15
47/47 [==============================] - 3s 69ms/step - loss: 1.9555e-04 - accuracy: 1.0000 - val_loss: nan - val_accuracy: 0.1667
Epoch 7/15
47/47 [==============================] - 3s 70ms/step - loss: 1.0828e-04 - accuracy: 1.0000 - val_loss: nan - val_accuracy: 0.1667
Epoch 8/15
47/

## 7. Fazer previsões no conjunto de teste

In [26]:
preds = model.predict(X_test_pad)
pred_labels = ["AI" if p > 0.5 else "Human" for p in preds.flatten()]

4/4 [==============================] - 0s 5ms/step


## 8. Exportar os resultados

In [ ]:
output_df = pd.DataFrame({
    "ID": ids,
    "Label": pred_labels
})
output_df.to_csv("../data/submissao2-grupo5-s1.csv", sep='\t', index=False)
print("✅ Ficheiro de submissão gerado: submissao2-grupo5-s1.csv")

✅ Ficheiro de submissão gerado: submissao2-grupo5-s1.csv
